<a href="https://colab.research.google.com/github/csar95/hugging-face-course/blob/main/workbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers